In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# EDA
import pandas as pd
import numpy as np

# Data Preprocessing 
from sklearn import preprocessing

# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Recommender System Imps
# Content Based Filtering 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Collaborative Based Filtering 
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# To work with text data 
import re
import string

In [ ]:
df = pd.read_csv('/kaggle/input/indian-food-and-its-recipes-dataset-with-images/cuisines.csv')
df.head()

In [ ]:
len(list(df['name'].unique()))

In [ ]:
df['course'].unique() # Categorical Data

In [ ]:
df['diet'].unique() # Categorical Data

In [ ]:
def remove_non_numeric(column):
    return column.apply(lambda x: re.sub(r'\D', '', str(x)))

df['prep_time'] = remove_non_numeric(df['prep_time'])

In [ ]:
df.head()

In [ ]:
def clean_column(column):
    """
    This function removes newline characters, tab characters, numeric values, and other symbols from a column
    of a pandas DataFrame.
    
    Args:
    column (pandas.Series): The column to clean.
    
    Returns:
    pandas.Series: A Series object containing the cleaned values from the input column.
    """
    # Remove newline and tab characters
    column = column.apply(lambda x: re.sub(r'[\n\t]', '', str(x)))
    
    # Remove numeric values and other symbols
    column = column.apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', str(x)))
    
    # Remove extra whitespace
    column = column.apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())
    
    return column

df['ingredients'] = clean_column(df['ingredients'])

In [ ]:
df.head()

In [ ]:
# General Description 
df.describe()

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['ingredients'])
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

In [ ]:
# Just considering the Food names from the dataframe
indices = pd.Series(df.index, index=df['name']).drop_duplicates()
indices

In [ ]:
class Profile:
    
    df = pd.read_csv('/kaggle/input/indian-food-and-its-recipes-dataset-with-images/cuisines.csv') # static variable
    
    def __init__(self,diet,cuisine,course,prep_time,ingredients):
        self.diet = diet
        self.cuisine = cuisine
        self.course = course
        self.prep_time = prep_time
        self.ingredients = ingredients
        self.df2 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df3 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df4 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df5 = pd.DataFrame(columns=list(Profile.df.columns))
        self.df6 = pd.DataFrame(columns=list(Profile.df.columns))
        
    def removestop(self,tokens):
        stop = set(stopwords.words('english'))
        file = open('/kaggle/input/stopword/stopwords.txt','r')
        l = list(file.read().split())
        stop = list(stop) +l
        l = [token for token in tokens if token not in stop]
        return l
            
    def inputs(self,diet,cuisine,course,prep_time,ingredients):
        
        if cuisine:
            self.df2 = Profile.df[Profile.df.cuisine.isin(cuisine)]
            self.df2 = self.df2.reset_index()
       
        if course:
            self.df2 = self.df2[self.df2.course.isin(course)]
            self.df2 = self.df2.reset_index()
      
        if diet:
            for i in range(self.df2.shape[0]):
                l = str(self.df2.loc[i,'diet']).split()
                
                for d in diet:
                    if d in l:
                        self.df4=self.df4.append(self.df2.loc[i])
 
        if prep_time:
            for i in range(self.df2.shape[0]):
                l = str(self.df2.loc[i,'prep_time']).split()
                for d in prep_time:
                    if d in l:
                        self.df5=self.df5.append(self.df2.loc[i])

        if ingredients:
            f = self.removestop(ingredients.split())
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'name']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6.append(Profile.df.loc[i])
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'ingredients']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6.append(df.loc[i])
            for i in range(Profile.df.shape[0]):
                n = [j.lower() for j in str(Profile.df.loc[i,'cuisine']).split()]
                for j in n:
                    for k in f:
                        if k==j:
                            self.df6=self.df6.append(Profile.df.loc[i])
            
        return self.df2,self.df3,self.df4,self.df5,self.df6
    
    def get_profile(self):
        df2,df3,df4,df5,df6 = self.inputs(self.diet,self.cuisine,self.course,self.prep_time,self.ingredients)
        
        
        df_merge = pd.concat([df2,df3,df4,df5,df6],axis=0).drop_duplicates(subset='name')
        df_merge = df_merge.filter(['name','image_url','instructions'])
        print(df_merge.shape)
        ans = df_merge.head(10)
     
        return ans
    
    
ob = Profile(['Vegetarian'],['Indian'],['Lunch'],['55'], 'I love Potatoes')

profile = ob.get_profile()
profile